In which I attempt to use the Camera controller to play nicely with the camera despite being agnostic about the units.



In [1]:
from SpheroController.tracablesphero import TraceableSphero

In [2]:
import sphero

# Controller Libraries
import time
import os
import json
from pprint import pprint
import math

In [3]:
# Tracking code
import cv2

# Modify from Simon Tordensky's Base 
from tracker.trackingfilter import FilterSpheroBlueCover, FilterGlow
from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker

from util import Vector2D

In [4]:
# Library written to help with tracking
import SpheroTeam

In [37]:
from SpheroTeam import normalize_angle, angle_between_points



In [5]:
from SpheroTeam.util import readJsonFile

In [6]:
# Config management
PROJ_ROOT = os.getcwd()
ROSTER = os.path.join(PROJ_ROOT, "roster.json")
roster = readJsonFile(ROSTER)


In [7]:
# Class for managing multiple spheros
manager = sphero.SpheroManager()

# Initialize Sphero manager with addresses of local spheros
manager._name_cache = roster

# Number of spheros to connect to
NBOTS = len(roster.keys())

In [8]:
# Add known spheros from roster to the manager
for bt_addr, bt_name in roster.iteritems():
    manager.add_sphero(bt_addr, bt_name)

In [9]:
devices = []
# Verify that these are the robots we are looking for
for name, device in manager._spheros.iteritems():
    print "{}: {}".format(name, device.bt_addr)
    devices.append(device) 

Sphero-RPB: 68:86:E7:05:12:26
Sphero-YPR: 68:86:E7:09:4E:74
Sphero-GRY: 68:86:E7:09:A9:28
Sphero-YYP: 68:86:E7:05:0C:4B
Sphero-RWR: 68:86:E7:09:A2:FE
Sphero-ORG: 68:86:E7:09:A6:FE
Sphero-RYR: 68:86:E7:05:19:AD


In [10]:
# Manually pick out the robots to include
# example: bots = [devices[4], devices[1]]
activeNames = ["Sphero-ORG",
               "Sphero-RWR"]




In [11]:
SpheroTeam.connect_team(bots)



Sphero-ORG try 0
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
Sphero-ORG try 1
Connected!
Sphero-RWR try 0
Connected!


In [12]:
# All robots start connnected
for bot in bots:
    
    bot.set_rgb(0,0,0)

NOTE! Removes wrong byte in start of header! Byte:  13


In [13]:
tracker = ColorTracker()
#Track Camera Size: 


tracker.image_size

(640, 480)

In [23]:
# Use blue because it's the most reliable color filter
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [24]:
# traceable_glow = TraceableObject("GLOW")
# traceable_glow.filter = FilterGlow()

In [25]:
# traceable_objects = tracker.track_objects(trackable_objects)

In [35]:
# For Debugging
def display_current_view(tracker):
    image = tracker.get_video_frame()
    cv2.imshow("img", image)
    cv2.waitKey(0)
    
    
# For Navigation
def get_bot_position(bot, traceable_object, tracker, samples=4, debug=False):  
    xSamples = []
    ySamples = []

    traceable_object.screen_size = tracker.image_size
     
    for sample in range(samples):
        image = tracker.get_video_frame()
        timestamp = time.time()
        
        if sample > 0: # ignore the first sample
            x, y = tracker._find_traceable_in_image(image, traceable_object) # side effect: adds mask to tracker  
            if x:
                xSamples.append(x)
            if y:
                ySamples.append(y)

            traceable_object.add_tracking(Vector2D(x, y), timestamp)
            cv2.waitKey(1) # is this really necessary

    if debug:      
        display_current_view(tracker)
    print "{} | {} ".format(xSamples, ySamples)
        
#     return traceable_object.pos.x, traceable_object.pos.y
    return (sum(xSamples) / len(xSamples)), (sum(ySamples) / len(ySamples))


def calibrate_bot_direction(bot, traceable_object_list, traceable_color, tracker, debug=False):
    """
        Routine for making all robots line up in the same direction
        
    """
#     traceable_object = traceable_object_list[0]
    TIMEOUT = 1500  # in MILLISECONDS
    
    
    bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
    bot.set_motion_timeout(TIMEOUT)
    startX, startY =  get_bot_position(bot, traceable_object_list[0], tracker)

#     traceable_object_list = tracker.track_object(traceable_object_list)
#     startX, startY = traceable_object_list[0].pos.x, traceable_object_list[0].pos.y
    cv2.waitKey(250)
    
    
    if (startX is None) or (startY is None):
        print("Error: Robot not in view")
        return -1
    
    print "Start ({},{})".format( startX, startY)

    bot.roll(60, 0)
    time.sleep(TIMEOUT / 1000) 

    endX, endY       =  get_bot_position(bot, traceable_object_list[0], tracker)
#     traceable_object_list = tracker.track_object(traceable_object_list)
#     endX, endY = traceable_object_list[0].pos.x, traceable_object_list[0].pos.y
      
    print "End   ({},{})".format(   endX, endY)
    offset = normalize_angle(angle_between_points(startX, startY, endX, endY))
    

    print "Angle {}".format(offset) 
#     bot.roll(50, offset)
#     time.sleep(1.5)
#     bot.set_heading(offset)
    
    
    # Turn lights off 
    
#     bot.set_rgb(0, 0, 0)
    
    return offset

In [27]:
# Print what the camera is currently seeing as a debugging step
# display_current_view(tracker)

In [28]:
# Check which bot is which
SpheroTeam.highlight_team(bots)



In [29]:
# Check which bots are ready
SpheroTeam.print_team_status(bots)


Sphero-ORG Battery OK | 8.05
Sphero-RWR Battery OK | 8.13


In [30]:
tracklist = [traceable_blue]


In [31]:
for bot in bots:
    bot.set_rgb(0, 0, 0)
    
    
    

In [32]:
# Video display
def display_tracking_window(traceable_object_list, exitKey = "q"):
    """
        Given a list of iterables, track all objects in that list.
        Press exitKey to quit the window
    """
    
    while(True):
        #     tracklist = tracker.track_objects(tracklist)

        tracker.track_objects(tracklist)
        cv2.waitKey(1)

        if cv2.waitKey(1) & 0xFF == ord(exitKey):
            break 
    cv2.destroyAllWindows()


In [38]:
offsets = []

for bot in bots:
    bot.set_rgb(0, 0, 255)
    time.sleep(2)
    offsets.append(calibrate_bot_direction(bot, tracklist, [0, 0, 255], tracker, True))
    bot.set_rgb(0,0,0)
    proceed = raw_input("Next Robot")
    
    

[444, 444, 445] | [319, 319, 319] 
Start (444,319)
[385, 382, 379] | [289, 288, 288] 
End   (382,288)
Angle 206.565051177
Enter anything to test next botasdf
[233, 233, 232] | [258, 258, 251] 
Start (232,255)
[268, 267, 262] | [319, 328, 314] 
End   (265,320)
Angle 63.083445383
Enter anything to test next bota


In [42]:
SpheroTeam.roll_sphero_team_synchronized(bots, 60, 0, offsets, 1500)


Dispatch Time 0.179999828339
Received unknown async msg! Header:  (255, 254, 5, 1)
Received unknown async msg! Header:  (255, 254, 5, 1)


Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 415, in _receiver
    header = self._receive_header()
  File "C:\Users\cyick\AppData\Roaming\Python\Python27\site-packages\sphero\core.py", line 299, in _receive_header
    first_byte = struct.unpack('B', raw_data)[0]
error: unpack requires a string argument of length 1

Exception in thread SpheroReceiverThread:
Traceback (most recent call last):
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\cyick\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "C:\Users\cyick\AppData\Roaming\Python\Pytho

In [ ]:
for bot in 

In [41]:
bots[0].roll(50, 206)


In [40]:
bot.roll(50, 63)









In [32]:
for i, bot in enumerate(bots):
    
    bot.roll(50, offsets[i])
    

In [81]:
bot.roll(50, 0)
time.sleep(1.5)
bot.roll(50, offset)
# bot.set_heading(0)




In [32]:
print traceable_blue.pos




(x: 281, y:156, deg:29.0372889653)


In [70]:
ts.device.roll(40, 180)




In [28]:
print traceable_blue.pos






(x: 444, y:201, deg:24.3563912528)


In [ ]:
for i in range(500):
    print get_bot_position(bot, traceable_blue, tracker, samples=4, debug=False)
    time.sleep(.5)
    

In [118]:
get_bot_position(bot, traceable_blue, tracker, samples=4, debug=False)








[183, 279, 279, 279] | [247, 241, 241, 241] 


(279, 241)

In [63]:
ts = TraceableSphero(bot)



In [53]:
print traceable_blue.pos



(x: 340, y:254, deg:36.7618944194)


In [19]:
ts.tracking_samples = []

image = tracker.get_video_frame()
x, y = tracker._find_traceable_in_image(image, traceable_blue)
ts.add_tracking(Vector2D(x, y), time.time())

print x
print y

143
371


In [20]:
print ts.pos



None


In [56]:
ts.calibrate_direction()

starts calibration
True


In [36]:
get_bot_position(bots[0], traceable_blue, tracker, debug=False)

[454, 454, 454, 454] | [202, 202, 202, 202] 


(454, 202)

[341] | [254] 
[340] | [254] 
Start (341,254)
End   (340,254)
Angle 180.0


180.0

In [85]:
bot.roll(50,270)


In [32]:
bot.roll(50, 135)


time.sleep(1.5)
bots[0].set_heading(0)





In [33]:
bots[0].roll(40, 180)



In [70]:



bots[0].roll(50, 0)


In [109]:
tracker._draw_masks()
cv2.waitKey(1)

Debugging Section

In [43]:
#Stream video live to set 

cam = tracker.cam


while(True):
    # Capture frame-by-frame
    ret, frame = cam.read()

    # Display the resulting frame
    cv2.imshow('frame', frame)
    
    #     cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()

In [ ]:
q